### Q1.

In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/slava/anaconda3/envs/llm_zoomcamp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"
print('Answer is: ', embedding_model.encode(user_question)[0])

Answer is:  0.078222625


### Q2.

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
documents = [ doc for doc in documents if doc['course']=="machine-learning-zoomcamp" ]
len(documents)

375

In [5]:
import numpy as np

embeddings = []
for doc in documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    embeddings.append(embedding_model.encode(qa_text))

X = np.array(embeddings)
print('Answer is: ', X.shape)

Answer is:  (375, 768)


### Q3.

In [6]:
v = embedding_model.encode(user_question)
scores = X.dot(v)

print('Answer is: ', scores.max())

Answer is:  0.6506574


### Q4.

In [7]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)


In [8]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [9]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [10]:
import time
start = time.time()

relevance_total = []
for q in ground_truth:
    doc_id = q['document']
    question = q['question']
    v = embedding_model.encode(question)
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

end = time.time()
print(end-start)

31.34316611289978


In [11]:
print('Answer is: ', hit_rate(relevance_total))

Answer is:  0.9398907103825137


### Q5.

In [12]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 


In [13]:
# Only question_text_vector

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [14]:
for idx, doc in enumerate(documents):
    qv_vector = X[idx]
    doc["question_text_vector"] = qv_vector
    

In [15]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [16]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)

knn = {
        "field": "question_text_vector",
        "query_vector": v,
        "k": 5,
        "num_candidates": 10000,
    }

search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

response = es_client.search(index=index_name, body=search_query)

In [17]:
print('Answer is: ', response["hits"]["hits"][0]["_source"]["id"])

Answer is:  ee58a693


### Q6. 

In [18]:
start = time.time()

relevance_total = []
for q in ground_truth:
    doc_id = q['document']
    question = q['question']
    v = embedding_model.encode(question)

    knn = {
            "field": "question_text_vector",
            "query_vector": v,
            "k": 5,
            "num_candidates": 10000,
        }
    
    search_query = {
            "knn": knn,
            "_source": ["text", "section", "question", "course", "id"]
        }
    
    results = es_client.search(index=index_name, body=search_query)
    results = [ hit['_source'] for hit in results['hits']['hits'] ]
    
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

end = time.time()
print(end-start)

58.17283844947815


In [19]:
print('Answer is: ', hit_rate(relevance_total))

Answer is:  0.9398907103825137


**Note:** ES has the same performance as Q4.VectorSearchEngine, but it appears to be almost two times slower